# EthioMart - Named Entity Recognition for Amharic E-commerce Data

## Overview

**EthioMart**, a growing hub for Telegram-based e-commerce in Ethiopia, aims to consolidate multiple independent e-commerce channels into a single centralized platform. With the increasing use of Telegram for business transactions, customers and vendors are currently spread across various channels, leading to challenges in product discovery, communication, and order management.

This project focuses on building an Amharic Named Entity Recognition (NER) system to extract important business entities—such as product names, prices, and locations—from the messages shared in these Telegram channels. The extracted data will be used to populate EthioMart's centralized database, providing a seamless and organized shopping experience for customers and a unified platform for vendors.

## Key Objectives

1. Real-time Data Extraction: Fetch data from various Ethiopian Telegram e-commerce channels.
1. Fine-tuning Large Language Models (LLMs): Adapt existing LLMs to accurately extract business entities like product names, prices, and locations from Amharic text.

## Install Necessary Packages

In [1]:
!pip install transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=fb8fa5071f49fe6b31f05f7d06330c26107aff5f8b44cc6e46c23e0748c3b5a8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# Import Necessary Libraries

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import accuracy_score, classification_report
import torch
from sklearn.model_selection import train_test_split
from collections import Counter

### Load the Dataset

In [3]:
def load_conll_dataset(file_path):
    sentences, labels = [], []
    sentence, label = [], []
    
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            if line.strip():  # If the line is not empty
                parts = line.strip().split()
                if len(parts) == 2:  # Ensure the line has exactly two components
                    token, tag = parts
                    sentence.append(token)
                    label.append(tag)
                else:
                    print(f"Skipping malformed line: {line.strip()}")
            else:
                if sentence:  # Append only if the sentence is not empty
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
    
    if sentence:  # Append any remaining sentence
        sentences.append(sentence)
        labels.append(label)
    
    return pd.DataFrame({"tokens": sentences, "ner_tags": labels})


### Split the dataset

In [4]:
# Function to split the CoNLL dataset into training and validation sets
def split_conll_dataset(conll_df, train_ratio=0.8):
    # Split the dataset into train and validation sets
    train_df, val_df = train_test_split(conll_df, train_size=train_ratio, random_state=42, shuffle=True)

    return train_df, val_df

# Example usage
file_path = "/kaggle/input/reduced-data/labeled_ner_data.txt"
conll_df = load_conll_dataset(file_path)  # Load the dataset
train_dataset, val_dataset = split_conll_dataset(conll_df)  # Split the dataset

# Check the sizes of the resulting datasets
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")


Training set size: 1072
Validation set size: 268


### Label Mapping

In [5]:
# Define label mapping
label_to_id = {
    "O": 0,  # Outside of entity
    "B-Product": 1,  # Beginning of a Product entity
    "I-Product": 2,  # Inside of a Product entity
    "B-PRICE": 3,  # Beginning of a Price entity
    "I-PRICE": 4,  # Inside of a Price entity
    "B-LOC": 5,  # Beginning of a Location entity
    "I-LOC": 6   # Inside of a Location entity
}

# Reverse mapping for predictions
id_to_label = {v: k for k, v in label_to_id.items()}

### Tokenize and Align Labels

In [6]:
def tokenize_and_align_labels(examples, tokenizer, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        # Replace any '0' (zero) with 'O' (uppercase letter O) and 'o' (lowercase) with 'O'
        label = ['O' if l in ['0', 'o'] else l for l in label]
        
        # Convert string labels to integers using label_to_id mapping
        label = [label_to_id[l] for l in label]  # Mapping the string NER tags to integers
        
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Padding token
            elif word_idx != previous_word_idx:  # First token of a word
                label_ids.append(label[word_idx])
            else:  # Non-first token of a word
                label_ids.append(-100 if not label_all_tokens else label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Train and Evaluate Model

In [7]:
# Function to fine-tune the model
def train_and_evaluate_model(model_name, train_dataset, val_dataset, label_list, batch_size=16, epochs=15):
    print(f"Training model: {model_name}")
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

    # Tokenize dataset
    # Passing tokenizer inside lambda function
    training_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)
    evaluation_dataset = val_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_dir=f"./logs_{model_name}",
        logging_steps=50
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=training_dataset,
        eval_dataset=evaluation_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation results for {model_name}:", eval_results)
    return eval_results

### Compute Metrics

In [8]:
def compute_metrics(pred):
    # Retrieve predictions and true labels
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Convert numeric labels back to their string names using id_to_label mapping
    true_labels = [[id_to_label[l] for l in label_row if l != -100] for label_row in labels]
    true_preds = [[id_to_label[p] for (p, l) in zip(pred_row, label_row) if l != -100] for pred_row, label_row in zip(preds, labels)]
    
    # Use seqeval to evaluate the performance
    report = classification_report(true_labels, true_preds)
    accuracy = accuracy_score(true_labels, true_preds)
    
    return {"accuracy": accuracy, "report": report}

### Compare Models

In [9]:
# Function to compare models
def compare_models(models, dataset, label_list):
    results = {}
    for model_name in models:
        eval_result = train_and_evaluate_model(model_name, dataset, label_list)
        results[model_name] = eval_result
    return results

In [10]:
# Load the labeled CoNLL dataset
#conll_df = load_conll_dataset("/kaggle/input/collection-ner/NER_Collection_data.txt")
#dataset = Dataset.from_pandas(conll_df)

### Count Labels in the Dataset

In [11]:
# Function to count each label in the dataset
def count_labels(dataset):
    all_labels = [label for labels in dataset['ner_tags'] for label in labels]
    label_counts = Counter(all_labels)
    
    # Print the counts for each label
    for label, count in label_counts.items():
        print(f"Label: {label}, Count: {count}")
    
    return label_counts

In [12]:
train_label_counts = count_labels(train_dataset)

Label: O, Count: 26629
Label: B-Product, Count: 1096
Label: I-Product, Count: 168
Label: B-LOC, Count: 3178
Label: I-LOC, Count: 28566
Label: B-Price, Count: 263
Label: I-Price, Count: 263


In [13]:
evaluation_label_counts = count_labels(val_dataset)

Label: B-Product, Count: 295
Label: O, Count: 7119
Label: B-LOC, Count: 829
Label: I-LOC, Count: 7388
Label: B-Price, Count: 67
Label: I-Price, Count: 67
Label: I-Product, Count: 44


#### Map labels to correct labels

In [14]:
# Function to map incorrect labels to correct labels
def map_labels(dataset):
    # Define the mapping from incorrect to correct labels
    label_mapping = {
        'B-PROD': 'B-Product',   # Map 'B-PROD' to 'B-Product'
        'B-PRODUCT': 'B-Product', # Map 'B-PRODUCT' to 'B-Product'
        'I-PRODUCT': 'I-Product', # Map 'I-PRODUCT' to 'I-Product'
        'B-Price': 'B-PRICE',    # Map 'B-Price' to 'B-PRICE'
        'I-Price': 'I-PRICE',    # Map 'I-Price' to 'I-PRICE'
        'IO': 'O'                # Map 'IO' to 'O'
    }
    
    # Replace the incorrect labels with the correct ones
    dataset['ner_tags'] = dataset['ner_tags'].apply(
        lambda tags: [label_mapping.get(tag, tag) for tag in tags]
    )
    
    return dataset


In [15]:
# Example usage:
train_df = map_labels(train_dataset)
train_dataset = Dataset.from_pandas(train_df)

# Verify the label counts after remapping
label_counts = count_labels(train_df)

Label: O, Count: 26629
Label: B-Product, Count: 1096
Label: I-Product, Count: 168
Label: B-LOC, Count: 3178
Label: I-LOC, Count: 28566
Label: B-PRICE, Count: 263
Label: I-PRICE, Count: 263


In [16]:
# Example usage:
val_df = map_labels(val_dataset)
val_dataset = Dataset.from_pandas(val_df)

# Verify the label counts after remapping
label_counts = count_labels(val_df)

Label: B-Product, Count: 295
Label: O, Count: 7119
Label: B-LOC, Count: 829
Label: I-LOC, Count: 7388
Label: B-PRICE, Count: 67
Label: I-PRICE, Count: 67
Label: I-Product, Count: 44


In [17]:
"""
# Function to save the dataset to storage
def save_dataset(dataset, file_path):
    
    Save the modified dataset to a specified file path in CSV format.
    
    Args:
        dataset (pd.DataFrame): The DataFrame containing the dataset.
        file_path (str): The file path where the dataset will be saved.
    
    dataset.to_csv(file_path, index=False)
    print(f"Dataset saved to {file_path}")

# Save the mapped dataset to a CSV file
save_dataset(conll_df, "preprocessed_conll_data.txt")

"""

'\n# Function to save the dataset to storage\ndef save_dataset(dataset, file_path):\n    \n    Save the modified dataset to a specified file path in CSV format.\n    \n    Args:\n        dataset (pd.DataFrame): The DataFrame containing the dataset.\n        file_path (str): The file path where the dataset will be saved.\n    \n    dataset.to_csv(file_path, index=False)\n    print(f"Dataset saved to {file_path}")\n\n# Save the mapped dataset to a CSV file\nsave_dataset(conll_df, "preprocessed_conll_data.txt")\n\n'

### List Models and Labels

In [18]:
"""
# List of entity labels 
label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC']

# Define models for comparison
models = [
    "xlm-roberta-base",  
    "bert-base-multilingual-cased",  
    "distilbert-base-multilingual-cased"  
]

# Compare models
results = compare_models(models, dataset, label_list)

"""

'\n# List of entity labels \nlabel_list = [\'O\', \'B-Product\', \'I-Product\', \'B-PRICE\', \'I-PRICE\', \'B-LOC\', \'I-LOC\']\n\n# Define models for comparison\nmodels = [\n    "xlm-roberta-base",  \n    "bert-base-multilingual-cased",  \n    "distilbert-base-multilingual-cased"  \n]\n\n# Compare models\nresults = compare_models(models, dataset, label_list)\n\n'

### Print Comparison Results

In [19]:
"""
# Print out comparison results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {result['eval_accuracy']}")
    print(result['eval_report'])

"""

'\n# Print out comparison results\nfor model_name, result in results.items():\n    print(f"Model: {model_name}")\n    print(f"Accuracy: {result[\'eval_accuracy\']}")\n    print(result[\'eval_report\'])\n\n'

In [20]:
#API = c912e406b425b51cb31ae3db26397612b381918d

### Fine-tune a Single Model at a Time

In [21]:
# Function to train and evaluate one model
def run_single_model(model_name, train_dataset, val_dataset, label_list):
    # Train and evaluate the model
    eval_result = train_and_evaluate_model(model_name, train_dataset, val_dataset, label_list)
    
    # Print the evaluation result for the model
    print(f"Model: {model_name}")
    print(f"Accuracy: {eval_result['eval_accuracy']}")
    print(eval_result['eval_report'])
    
    return eval_result


In [22]:
# List of entity labels 
label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC']

# Define the model to run
model_name = "xlm-roberta-base"

# Run and evaluate the model
eval_result = run_single_model(model_name, train_dataset, val_dataset, label_list)


Training model: xlm-roberta-base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Report
1,0.589400,0.077598,0.980091,precision recall f1-score support LOC 0.99 0.93 0.96 1857 PRICE 0.00 0.00 0.00 82 Product 0.94 0.50 0.65 640 micro avg 0.98 0.79 0.88 2579 macro avg 0.64 0.48 0.54 2579 weighted avg 0.95 0.79 0.85 2579
2,0.087100,0.032283,0.991793,precision recall f1-score support LOC 0.99 0.97 0.98 1857 PRICE 0.96 0.91 0.94 82 Product 0.71 0.88 0.79 640 micro avg 0.91 0.94 0.93 2579 macro avg 0.89 0.92 0.90 2579 weighted avg 0.92 0.94 0.93 2579
3,0.029600,0.016955,0.994485,precision recall f1-score support LOC 0.98 0.98 0.98 1857 PRICE 0.95 0.99 0.97 82 Product 0.80 0.91 0.85 640 micro avg 0.93 0.96 0.95 2579 macro avg 0.91 0.96 0.93 2579 weighted avg 0.94 0.96 0.95 2579
4,0.018600,0.011398,0.995650,precision recall f1-score support LOC 0.99 0.98 0.98 1857 PRICE 0.96 0.96 0.96 82 Product 0.83 0.91 0.87 640 micro avg 0.95 0.96 0.95 2579 macro avg 0.93 0.95 0.94 2579 weighted avg 0.95 0.96 0.95 2579
5,0.014300,0.010900,0.996634,precision recall f1-score support LOC 0.99 0.98 0.99 1857 PRICE 0.96 1.00 0.98 82 Product 0.90 0.93 0.91 640 micro avg 0.96 0.97 0.97 2579 macro avg 0.95 0.97 0.96 2579 weighted avg 0.97 0.97 0.97 2579
6,0.008900,0.006830,0.997851,precision recall f1-score support LOC 0.98 0.99 0.98 1857 PRICE 0.96 1.00 0.98 82 Product 0.97 0.96 0.97 640 micro avg 0.98 0.98 0.98 2579 macro avg 0.97 0.98 0.98 2579 weighted avg 0.98 0.98 0.98 2579
7,0.005300,0.005400,0.998524,precision recall f1-score support LOC 0.99 0.99 0.99 1857 PRICE 0.96 1.00 0.98 82 Product 0.97 0.98 0.98 640 micro avg 0.98 0.99 0.99 2579 macro avg 0.97 0.99 0.98 2579 weighted avg 0.98 0.99 0.99 2579
8,0.003800,0.004739,0.998524,precision recall f1-score support LOC 0.99 0.99 0.99 1857 PRICE 0.98 1.00 0.99 82 Product 0.98 0.98 0.98 640 micro avg 0.98 0.99 0.99 2579 macro avg 0.98 0.99 0.98 2579 weighted avg 0.98 0.99 0.99 2579
9,0.003100,0.004230,0.998809,precision recall f1-score support LOC 0.99 0.99 0.99 1857 PRICE 0.96 1.00 0.98 82 Product 0.97 0.99 0.98 640 micro avg 0.99 0.99 0.99 2579 macro avg 0.98 0.99 0.98 2579 weighted avg 0.99 0.99 0.99 2579
10,0.002300,0.004322,0.998861,precision recall f1-score support LOC 0.99 0.99 0.99 1857 PRICE 0.96 1.00 0.98 82 Product 0.97 0.99 0.98 640 micro avg 0.99 0.99 0.99 2579 macro avg 0.98 0.99 0.99 2579 weighted avg 0.99 0.99 0.99 2579


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "              precision    recall  f1-score   support

         LOC       0.99      0.93      0.96      1857
       PRICE       0.00      0.00      0.00        82
     Product       0.94      0.50      0.65       640

   micro avg       0.98      0.79      0.88      2579
   macro avg       0.64      0.48      0.54      2579
weighted avg       0.95      0.79      0.85      2579
" of type <class 'str'> for key "eval/report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "              precision    recall  f1-score   support

         LOC       0.99      0.

Trainer is attempting to log a value of "              precision    recall  f1-score   support

         LOC       0.99      0.99      0.99      1857
       PRICE       0.96      1.00      0.98        82
     Product       0.99      0.99      0.99       640

   micro avg       0.99      0.99      0.99      2579
   macro avg       0.98      1.00      0.99      2579
weighted avg       0.99      0.99      0.99      2579
" of type <class 'str'> for key "eval/report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Evaluation results for xlm-roberta-base: {'eval_loss': 0.004102285951375961, 'eval_accuracy': 0.9990938511326861, 'eval_report': '              precision    recall  f1-score   support\n\n         LOC       0.99      0.99      0.99      1857\n       PRICE       0.96      1.00      0.98        82\n     Product       0.99      0.99      0.99       640\n\n   micro avg       0.99      0.99      0.99      2579\n   macro avg       0.98      1.00      0.99      2579\nweighted avg       0.99      0.99      0.99      2579\n', 'eval_runtime': 4.2885, 'eval_samples_per_second': 62.493, 'eval_steps_per_second': 3.964, 'epoch': 15.0}
Model: xlm-roberta-base
Accuracy: 0.9990938511326861
              precision    recall  f1-score   support

         LOC       0.99      0.99      0.99      1857
       PRICE       0.96      1.00      0.98        82
     Product       0.99      0.99      0.99       640

   micro avg       0.99      0.99      0.99      2579
   macro avg       0.98      1.00      0.99    